In [ ]:
# Works best with jupyter-notebook

In [ ]:
%matplotlib notebook 
#%matplotlib widget 
# https://ipython.readthedocs.io/en/stable/interactive/magics.html
import numpy as np
import math
from collections import namedtuple

from spatialmath import *
from spatialmath.base import *

from roboticstoolbox import *
import roboticstoolbox.tools.trajectory as tr

import matplotlib.pyplot as plt

np.set_printoptions(linewidth=100, formatter={'float': lambda x: f"{x:8.4g}" if abs(x) > 1e-10 else f"{0:8.4g}"})

# Lecture 4.4

Let us first look at how to create a 1D trajectory with multiple via points

In [ ]:
#Set a first point of 10, a last point of 30 and via points of 40, 10 and the last point
first = np.array(10)
last  = np.array(10)

via = np.array([
    [10],
    [20],
    [10]
    ])

print("First: ", first, "\nLast: ", last, "\nVia Points: \n", via.squeeze())


Compute a trajectory with via points using the toolbox function **mstraj()**.

Params: 
 - via points (one per row)
 - time step <secs> (dt)
 - acceleration time <secs> (tacc)
 - max speed, defaults to none <units/sec> (qd)
 - max time of each motion segment <secs> (tseg)
 - initial via point position
 - initial via point velocity
 - returns trajectory plus extra info <tuple> as a named tuple
    
The return value is a namedtuple (named ``mstraj``) with elements:
    
        - ``t``  the time coordinate as a numpy ndarray, shape=(K,)
        - ``q``  the axis values as a numpy ndarray, shape=(K,N)
        - ``arrive`` a list of arrival times for each segment
        - ``info`` a list of named tuples, one per segment that describe the
          slowest axis, segment time,  and time stamp
        - ``via`` the passed set of via points
    
    The  trajectory proper is (``traj.t``, ``traj.q``).  The trajectory is a matrix has one row per time step, and one column per axis.


In [ ]:
speed = 1   # velocity is 1 unit/second
dt    = 1 # time interval 0.1 seconds
tacc  = 0   # acceleration time 2 seconds

out = tr.mstraj(via, dt, tacc, qdmax=speed, q0=first)

In [ ]:
print(type(out.t))

print(type(out.arrive))

print(type(out.via))


In [ ]:
print("The trajectory lasts a total of ", out.t[-1], " secs.")

print("The segments times are as follows \n", out.arrive)
 
print("My via points are: \n", out.via.squeeze())

#print("Print out every nth point of the trajectory \n", out.q[0::10])
#out.info # Useful with more axis

Plot the trajectory

In [ ]:
fig1=plt.figure(1);
plt.plot(out.t, out.q);
plt.xlabel('time');
plt.ylabel('unit');

Increasing the acceleration time from 0 to 2 seconds, increases the duration of our acceleration (slows down the trajectory).

So, to achieve starting/final via points smoothly, the trajectory moves away from the via points.

In [ ]:
# control precision
np.set_printoptions(precision=2)

speed = 1   
dt    = 1 
tacc  = 2 

out = tr.mstraj(via, dt, tacc, qdmax=speed, q0=first)
plt.plot(out.t, out.q)
plt.xlabel('time')
plt.ylabel('unit')

In [ ]:
print("The trajectory lasts a total of ", out.t[-1], " secs.")

print("The four segments arrive at the following times: \n", out.arrive)
 
print("My via points are: \n", out.via.squeeze())

print("Printint out every 20th point of the trajectory \n", out.q[0::20])
# out.info # Useful with more axis

In [ ]:
#Increasing acceleration time from 2 to 8 seconds
speed = 1   
dt    = 1 
tacc  = 8 

out=tr.mstraj(via, dt, tacc, qdmax=speed, q0=first)

In [ ]:
#plot
#fig3=plt.figure(3)
plt.plot(out.t, out.q)
plt.xlabel('time')
plt.ylabel('unit')

If we increase the velocity, the amount of time taken to execute the trajectory is reduced.

In [ ]:
speed = 4   
dt    = 1 
tacc  = 8 

out=tr.mstraj(via, dt, tacc, qdmax=speed, q0=first)

#plot
#fig5=plt.figure(5)
plt.plot(out.t, out.q)
plt.xlabel('time')
plt.ylabel('unit')

We can set the duration of each segment instead of the velocity. 

The time duration of each segment is passed via a vector to the function. 

Segment 1: 10 secs

Segment 2: 20 secs

In [ ]:
dt   = 0.1 # time interval 0.1 seconds
tacc = 2 # acceleration time 4 seconds
print(via)
out=tr.mstraj(via, dt, tacc, tsegment=[0, 10, 20], q0=first)

In [ ]:
#plot
fig6=plt.figure(6)
plt.plot(out.t, out.q)
plt.xlabel('time')
plt.ylabel('unit')